In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import datasets, transforms
from torchvision.datasets import mnist, CIFAR10
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

from sklearn import metrics
import xgboost

import time
import torch.nn.functional as TF
import torch.optim as optim
import os
import math
import matplotlib.pyplot as plt
import pickle

torch.set_num_threads(1)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
# (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616]),
])
test_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616]),
])

train_dataset = CIFAR10(root='data/CIFAR10/train', train=True,
                              download=True,transform=transform)
test_dataset = CIFAR10(root='data/CIFAR10/test', train=False,
                             download=True, transform=test_transform)

x_tr = train_dataset.data
y_tr = np.array(train_dataset.targets)
x_tr = x_tr.reshape((-1, 32*32*3))

x_te = test_dataset.data
y_te = np.array(test_dataset.targets)
x_te = x_te.reshape((-1, 32*32*3))

all_x = np.concatenate((x_tr,x_te),axis=0)
all_y = np.concatenate((y_tr,y_te),axis=0)
train_length = len(x_tr)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
pca = PCA(n_components=20)
all_x_pca = pca.fit_transform(all_x)
x_tr = all_x_pca[:train_length]
x_te = all_x_pca[train_length:]

print(x_tr.shape)
print(x_te.shape)

(50000, 20)
(10000, 20)


In [4]:
SAVE_DIR = 'Models/'
for SEED in range(5):
    np.random.seed(SEED)
    all_index = np.arange(len(x_tr))
    np.random.shuffle(all_index)
    train_index = all_index[0:int(0.9*len(x_tr))]
    val_index = all_index[int(0.9*len(x_tr)):]

    train_x = x_tr[train_index]
    train_y = y_tr[train_index]
    val_x = x_tr[val_index]
    val_y = y_tr[val_index]
    # Train model
    xgb_clf = xgboost.XGBClassifier(use_label_encoder=False)
#     xgb_clf = xgboost.XGBClassifier(use_label_encoder=False,n_estimators=100,max_depth=25,max_leaves=30,learning_rate=0.01)
    xgb_clf.fit(train_x, train_y)
    # Save model
    save_path = SAVE_DIR + 'CIFAR10_XGBoost_SEED_%d.pkl'%SEED
    pickle.dump(xgb_clf, open(save_path, 'wb'))
    # Load model
    xgb_val_clf = pickle.load(open(save_path, 'rb'))
    
    y_pred = xgb_val_clf.predict(val_x)
    score = metrics.accuracy_score(val_y, y_pred)
    print('SEED:%d,Accuracy Score:%.4f%%'%(SEED,100*score))

[04:16:54] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
SEED:0,Accuracy Score:45.7400%
[04:17:27] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
SEED:1,Accuracy Score:45.1600%
[04:17:59] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
SEED:2,Accuracy Score:45.3600%
[04:18:31] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was ch

In [5]:
SAVE_DIR = 'Models/'
for SEED in range(5):
    np.random.seed(SEED)
    all_index = np.arange(len(x_tr))
    np.random.shuffle(all_index)
    train_index = all_index[0:int(0.9*len(x_tr))]
    val_index = all_index[int(0.9*len(x_tr)):]

    train_x = x_tr[train_index]
    train_y = y_tr[train_index]
    val_x = x_tr[val_index]
    val_y = y_tr[val_index]
    val_y_onehot = np.eye(10)[val_y]

    
    save_path = SAVE_DIR + 'CIFAR10_XGBoost_SEED_%d.pkl'%SEED
    
    xgb_val_clf = pickle.load(open(save_path, 'rb'))
    
    y_pred = xgb_val_clf.predict(val_x)
    y_prob = xgb_val_clf.predict_proba(val_x)
    
    score = metrics.accuracy_score(val_y, y_pred)
    F1_score = metrics.f1_score(val_y, y_pred, average='weighted')
    ROC_AUC_score = metrics.roc_auc_score(val_y_onehot, y_prob,average='samples',multi_class='ovo')
    print('SEED:%d'%(SEED))
    print('Accuracy Score:', score)
    print('F1 Score:', F1_score)
    print('ROC AUC Score:', ROC_AUC_score)
    print('Average score:%.4f'%((score+F1_score+ROC_AUC_score)/3))

SEED:0
Accuracy Score: 0.4574
F1 Score: 0.4542353443341773
ROC AUC Score: 0.8286
Average score:0.5801
SEED:1
Accuracy Score: 0.4516
F1 Score: 0.44935228230665664
ROC AUC Score: 0.8334666666666668
Average score:0.5781
SEED:2
Accuracy Score: 0.4536
F1 Score: 0.451574959849309
ROC AUC Score: 0.8348222222222222
Average score:0.5800
SEED:3
Accuracy Score: 0.4548
F1 Score: 0.4518173426238597
ROC AUC Score: 0.8330888888888889
Average score:0.5799
SEED:4
Accuracy Score: 0.4564
F1 Score: 0.4540931025325998
ROC AUC Score: 0.8329111111111112
Average score:0.5811


In [13]:
SAVE_DIR = 'Models/'
SEED = 4

y_te_onehot = np.eye(10)[y_te]
save_path = SAVE_DIR + 'CIFAR10_XGBoost_SEED_%d.pkl'%SEED

xgb_val_clf = pickle.load(open(save_path, 'rb'))

y_pred = xgb_val_clf.predict(x_te)
y_prob = xgb_val_clf.predict_proba(x_te)

score = metrics.accuracy_score(y_te, y_pred)
F1_score = metrics.f1_score(y_te, y_pred, average='weighted')
ROC_AUC_score = metrics.roc_auc_score(y_te_onehot, y_prob,multi_class='ovo')
print('SEED:%d'%(SEED))
print('Accuracy Score:', score)
print('F1 Score:', F1_score)
print('ROC AUC Score:', ROC_AUC_score)

SEED:4
Accuracy Score: 0.4568
F1 Score: 0.4541473383937634
ROC AUC Score: 0.8585634666666667
